In [14]:
import requests
import re
import io
import datetime
import time
from progressbar import *
import os

In [15]:
def download(keywords,category='gr-qc'):
    #create a directory in the name of date and category
    dir_path = str(datetime.date.today())+'/'+category
    if os.path.exists(dir_path):
        print('Already exist!')
    else:
        os.makedirs(dir_path)
    url = 'https://arxiv.org/list/%s/new' %category
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50'}
    html = requests.get(url).content #add .decode('utf-8') for python 3 
    titles = re.findall(r'<div class="list-title mathjax">(.*?)</div>',html,re.S)
    links = re.findall(r'<span class="list-identifier">(.*?)</span>',html,re.S)
    articles = []
    for i in range(len(titles)):
        title = titles[i][titles[i].rfind('>')+2:].strip() 
        link = 'https://arxiv.org%s' %links[i][links[i].find('/pdf'):links[i].find('/pdf')+15]
        articles.append({'Title':title,'Link':link})
    widgets = ['Progress: ',Percentage(), ' ', Bar('#'),' ', Timer(),  
               ' ', ETA(), ' ', FileTransferSpeed()]  
    pbar = ProgressBar(widgets=widgets, maxval=10*len(articles)).start()
    i = 0
    j = 0
    for article in articles:
        for keyword in keywords:
            if keyword in map(lambda x:x.lower(),article['Title'].split(' ')):
                art_html = requests.get(article['Link'])
                fp = io.open('%s/%s.pdf'%(dir_path,article['Title']),'wb')
                fp.write(art_html.content)
                fp.close()
                j += 1
                break
            else:
                pass
        pbar.update(10 * i + 1)
        i += 1

    pbar.finish()
    print('*'*60)
    print('{} artilce(s) have(has) been downloaded, enjoy yourself!'.format(j))

In [16]:
keywords = ['gravitational','wave','black','hole','waveform']
category1 = 'gr-qc'
category2 = 'astro-ph'
if __name__ == '__main__':
    %time download(keywords=keywords,category=category2)

**************************************************
12 artilce(s) have(has) been downloaded, enjoy yourself!
CPU times: user 1.2 s, sys: 405 ms, total: 1.6 s
Wall time: 1min 45s
